Used Car Exploratory Data Analysis

In this jupyter notebook, the libraries pandas, streamlit, plotly.express and altair are used. Pandas is used as a means for exploratory data analysis to dechiper the best deal on cars when a variable is compared against the price of the car. Streamlit is used to run the application from the command line on a local machine. Plotly.express is used to visualize data in graphs. 

In [79]:
import pandas as pd
import streamlit as sl
import plotly.express as px
import altair as alt

df = pd.read_csv('vehicles_us.csv')
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [80]:
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d')
df['model_year'] = pd.to_datetime(df['model_year'], format = '%Y')
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  datetime64[ns]
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](2), float64(3), int64(2), object(6)
memory usage: 5.1+ MB


In [81]:
df[df.duplicated()]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed


In [82]:
#fill missing values in every column
df['is_4wd'] = df['is_4wd'].fillna('No')
df['is_4wd'] = df['is_4wd'].replace({1: 'Yes'})
#fill missing price values with the mean
price_mean = df['price'].mean()
df['price'] = df['price'].fillna(price_mean)
#replace values of 1 with the mean of the price
df['price'] = df['price'].replace(1, price_mean)
#forward fill the years
df['model_year'] = df['model_year'].ffill()

df['paint_color'] = df['paint_color'].fillna('None Provided')

#find the average of the mileage for all cars and use that number to fill the missing values
odo_mean = df['odometer'].mean()
df['odometer'] = df['odometer'].fillna(odo_mean)

#find the average of the cylinders and use that number to fill the missing values
cylinder_mean = df['cylinders'].mean()
df['cylinders'] = df['cylinders'].fillna(cylinder_mean)

#split the model_name column into two new string columns 'manufacturer' and 'model'
df[['manufacturer', 'model_name']] = df['model'].str.split(' ', n=1, expand=True)
df = df.drop(columns=['model'])
df.head()
df.tail()



,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer,model_name
51520,9249.0,2013-01-01,like new,6.0,gas,88136.000000,automatic,sedan,black,No,2018-10-03,37,nissan,maxima
51521,2700.0,2002-01-01,salvage,4.0,gas,181500.000000,automatic,sedan,white,No,2018-11-14,22,honda,civic
51522,3950.0,2009-01-01,excellent,4.0,gas,128000.000000,automatic,sedan,blue,No,2018-11-15,32,hyundai,sonata
51523,7455.0,2013-01-01,good,4.0,gas,139573.000000,automatic,sedan,black,No,2018-07-02,71,toyota,corolla
51524,6300.0,2014-01-01,good,4.0,gas,115553.461738,automatic,sedan,None Provided,No,2018-06-05,10,nissan,altima


In [83]:
#show how many cars are on the website by manufacturer
#list in descending order
organized_df = filtered_df.groupby('manufacturer').size().reset_index(name='count')


organized_df = organized_df.sort_values(by='count', ascending=False)

model_counts = filtered_df['manufacturer'].value_counts()

model_dfs = {}

for manufacturer in model_counts.index:
    model_dfs[manufacturer] = organized_df[organized_df['manufacturer'] == manufacturer].reset_index(drop=True)

print("Manufacturer Counts:")
model_counts




Manufacturer Counts:


manufacturer
ford             4361
chevrolet        3459
toyota           2049
honda            1242
ram              1191
jeep             1079
gmc               936
nissan            840
subaru            487
dodge             420
hyundai           387
chrysler          257
volkswagen        244
kia               166
cadillac          142
acura             104
bmw                99
buick              95
mercedes-benz      24
Name: count, dtype: int64

In [84]:
#make a dataset to show all of the cars that have been on the market for over a month and have over 100000 miles
(df['odometer'].unique())
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')
df['days_listed'] = pd.to_numeric(df['days_listed'], errors='coerce')
filtered_df = df[(df['odometer'] > 100000) & (df['days_listed'] > 31)]
filtered_df.head()

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer,model_name
2,5500.0,2013-01-01,like new,4.0,gas,110000.000000,automatic,sedan,red,No,2019-02-07,79,hyundai,sonata
7,15990.0,2013-01-01,excellent,6.0,gas,109473.000000,automatic,SUV,black,Yes,2019-01-07,68,honda,pilot
10,19500.0,2011-01-01,excellent,8.0,gas,128413.000000,automatic,pickup,black,Yes,2018-09-17,38,chevrolet,silverado 1500
12,18990.0,2012-01-01,excellent,8.0,gas,140742.000000,automatic,pickup,None Provided,Yes,2019-04-02,37,ram,1500
15,17990.0,2013-01-01,excellent,8.0,gas,115553.461738,automatic,pickup,red,Yes,2018-05-15,111,ram,1500


In [95]:
#make a graph that shows which manufacturer has the most cars listed on the site that has over 100000 miles and on the market for over a month
filtered_df = df[(df['odometer'] > 100000) & (df['days_listed'] > 31)]

fig = px.histogram(filtered_df, x='manufacturer',
                   title='Number of Cars with Over 100,000 Miles And on Market for over a Month by Manufacturer',
                   labels={'manufacturer': 'Car Manufacturer'},
                   color_discrete_sequence=['pink'])

fig.update_layout(yaxis_title='Number of Cars', xaxis_title='Car Manufacturer')

fig.show()

this graph shows that ford has the most cars on the site with over 100000 miles on it and has been on the market for over a month with 4,361 cars while the least amount of cars with these conditions are mercedes with only 24 cars.

In [96]:
#find the ten most and least expensive cars
df['price'] = pd.to_numeric(df['price'], errors='coerce')

sorted_df = df.sort_values(by='price')

least_expensive = sorted_df.head(10)[['model_name', 'price' , 'odometer' , 'days_listed']]

most_expensive = sorted_df.tail(10)[['model_name', 'price' , 'odometer' , 'days_listed']]

print("\nTen Least Expensive Cars:\n")
print(least_expensive.to_string(index=False))

print("\nTen Most Expensive Cars:\n")
print(most_expensive.to_string(index=False))



Ten Least Expensive Cars:

model_name  price      odometer  days_listed
   liberty    3.0 115553.461738           22
    sienna    5.0 115553.461738            5
      f250    6.0 173500.000000           53
  forester    9.0    119.000000           12
     prius   10.0 101000.000000           29
  corvette   11.0  30300.000000           25
     prius   12.0    106.000000           19
   charger   12.0  53200.000000           34
     versa   12.0 115553.461738           24
      f250   15.0 191000.000000           44

Ten Most Expensive Cars:

   model_name    price      odometer  days_listed
       tundra 145000.0 140000.000000           29
sierra 2500hd 175000.0 149000.000000           57
        f-150 189000.0 151248.000000          102
        f-150 189000.0 151248.000000           72
        f-150 189000.0 151248.000000           28
        f-150 189000.0 115553.461738           42
        f-150 189000.0 151248.000000           56
        f-150 189000.0 151248.000000           21


In [97]:
#find the least expensive cars with more than 100000 miles and on the market for more than a month
least_expensive_filtered = least_expensive[
    (least_expensive['odometer'] > 100000) & 
    (least_expensive['days_listed'] > 31)
]

least_expensive_filtered.head(10)


,model_name,price,odometer,days_listed
31263,f250,6.0,173500.0,53
9214,f250,15.0,191000.0,44


In [88]:
#find the most expensive cars with more than 100000 miles and on the market for more than a month
most_expensive_filtered = most_expensive[
    (most_expensive['odometer'] > 100000) & 
    (most_expensive['days_listed'] > 31)
]

most_expensive_filtered.head(10)


,model_name,price,odometer,days_listed
34206,sierra 2500hd,175000.0,149000.000000,57
33434,f-150,189000.0,151248.000000,102
27375,f-150,189000.0,151248.000000,72
30634,f-150,189000.0,115553.461738,42
1309,f-150,189000.0,151248.000000,56
11359,2500,300000.0,115553.461738,39


In [89]:
df.head()

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer,model_name
0,9400.0,2011-01-01,good,6.0,gas,145000.000000,automatic,SUV,None Provided,Yes,2018-06-23,19,bmw,x5
1,25500.0,2011-01-01,good,6.0,gas,88705.000000,automatic,pickup,white,Yes,2018-10-19,50,ford,f-150
2,5500.0,2013-01-01,like new,4.0,gas,110000.000000,automatic,sedan,red,No,2019-02-07,79,hyundai,sonata
3,1500.0,2003-01-01,fair,8.0,gas,115553.461738,automatic,pickup,None Provided,No,2019-03-22,9,ford,f-150
4,14900.0,2017-01-01,excellent,4.0,gas,80903.000000,automatic,sedan,black,No,2019-04-02,28,chrysler,200


In [90]:
#make a graph to find out which car brand on average is the most expensive
#get the average price of the manufacturers of cars
average_price = df.groupby('manufacturer')['price'].mean().reset_index()
#graph it
fig = px.bar(average_price, 
             x='manufacturer', 
             y='price', 
             title='Average Price of Cars by Manufacturer',
             labels={'price': 'Average Price'},
             color='price',
             color_continuous_scale='Blues')

fig.show()

This graph shows the average prices of cars by manufacturer on the site. The darker blue bars represent the most expensive average price while the lighter blue bars show less expensive average prices. The graph shows that mercedes-benz cars are the most expensive on the site, average a price of $34,900. The least expensive brand of car hyundai, costing $7,256.99, which beats out volkswagon for cheapest car by less than a dollar, costing $7,257.40.

In [91]:
#aggregate the data to get group the manufacturer and condition
#aggregate the data to get the average prices of those car conditions by manufacturer

aggregated_df = filtered_df.groupby(['manufacturer', 'condition'], as_index=False)['price'].mean()

#make an interactive graph with this new aggregatred data
condition_colors = {
    'excellent': 'pink',
    'like new': 'purple',
    'good': 'orange',
    'fair': 'yellow',
    'salvage': 'green',
    'new': 'red'
}
fig = px.bar(aggregated_df, x='manufacturer', y='price', color='condition',
             title='Average Price by Car Condition and Manufacturer',
             labels={'price': 'Average Price', 'manufacturer': 'Car Manufacturer'},
             color_discrete_map=condition_colors,  
             barmode='group')  

fig.update_layout(yaxis_title='Average Price', xaxis_title='Car Manufacturer')

fig.show()

This graph shows the average price of cars per manufacturer by the condition of the car listed on the site. Along the right hand side the user can click on conditions to exclude from the list to see exactly what condition of car they would like to view. the most expensive car on average is a mercedes that is in excellent condition with an average price of $16,836 while the least expensive car and condition is a salvage condition gmc with an average price of $700. What was interesting in the graph was that it showed there was no new cadillacs, bmws, or kias. the next code verifies if this is true

In [92]:
#check if there are in fact no new cadillacs in the dataset
cadillac_cars = df[df['manufacturer'].str.lower() == 'cadillac']

if any(cadillac_cars['condition'] == 'new'):
    print("There are new Cadillacs in the dataset.")
else:
    print("There are no new Cadillacs in the dataset.")

#check if there are in fact no new bmws in the dataset

bmw_cars = df[df['manufacturer'].str.lower() == 'bmw']

if any(bmw_cars['condition'] == 'new'):
    print("There are new bmws in the dataset.")
else:
    print("There are no new bmws in the dataset.")

#check if there are in fact no new kias in the dataset
kia_cars = df[df['manufacturer'].str.lower() == 'kia']

if any(kia_cars['condition'] == 'new'):
    print("There are new kias in the dataset.")
else:
    print("There are no new kias in the dataset.")



There are no new Cadillacs in the dataset.
There are no new bmws in the dataset.
There are no new kias in the dataset.


In [93]:

#creating seperate datasets with diesel and gas cars
diesel_cars = df[df['fuel'] == 'diesel']
gas_cars = df[df['fuel'] == 'gas']

#counting how many diesel cars by manufacturer
diesel_count = diesel_cars['manufacturer'].value_counts().reset_index()
diesel_count.columns = ['manufacturer', 'Diesel']

#counting how many gas cars by manufacturer
gas_count = gas_cars['manufacturer'].value_counts().reset_index()
gas_count.columns = ['manufacturer', 'Gas']

#combining the counts by manufacturer
combined_counts = pd.merge(diesel_count, gas_count, on='manufacturer', how='outer').fillna(0)

#transferring all counts to integers
combined_counts['Diesel'] = combined_counts['Diesel'].astype(int)
combined_counts['Gas'] = combined_counts['Gas'].astype(int)

#making the graph
fig = px.bar(combined_counts,
             x='manufacturer',
             y=['Diesel', 'Gas'],
             title='Number of Cars by Manufacturer and Fuel Type',
             labels={'manufacturer': 'Manufacturer', 'value': 'Number of Cars'},
             color_discrete_sequence=['#1f77b4', '#ff7f0e'],
             )
fig.show()

this graph compares the number of cars listed on the site by fuel type. Ford has the most amount of gas cars with 11,229 cars and also has the most diesel cars listed with 1,379.

In [94]:
#making a graph that compares the prices of cars based on fuel types
average_prices = df.groupby('fuel')['price'].mean().reset_index()

fig = px.bar(average_prices, 
             x='fuel', 
             y='price', 
             title='Average Price of Cars by Fuel Type',
             labels={'fuel': 'Fuel Type', 'price': 'Average Price'},
             color='fuel',
             color_discrete_sequence=['red', 'blue' , 'green' , 'yellow' , 'pink'])

fig.show()

this graph shows that diesel cars are the most expensive on the site and electric cars are the cheapest. The "other" column accounts of the average price of cars with missing fuel type values. 

Conclusion:

In this exploratory data analysis, the aim was to compare how the average price of the cars change based on fuel type, condition and manufacturer. Through graphing different conditions against the price users will be able to see that the most expensive cars tend to be diesel cars, with the most expensive cars on the site being mercedes-benz. The cheapest cars tended to be those the 'salvaged' condition, showing that while they are the cheapest, they might not be able to run very well. The data also shows that Ford has the most cars with the least desirable conditions for customers (more than 100000 miles and on the market for over a month) showing that it may be best to place deals on fords to make room for more inventory, especially since it seems to be that fords have the most cars listed on the site. 